In [ ]:
import os
import numpy as np
import plotly.graph_objects as go 
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime, timezone
import json  

### Извлечение данных

In [ ]:

# Check all file names in data directory
dir_path = "C:\\Users\\picni\\OneDrive\\Рабочий стол\\Работа\\opto_ident\\Данные\\204\\Data"
files_indir_names = []
for path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, path)):
        files_indir_names.append(dir_path + "\\" + path)
data = {}
# Iterate all files in data directory and add <data> information 
for file_name in tqdm(files_indir_names):
    with open(file_name, 'r') as f:
        file = f.read() 
    soup = BeautifulSoup(file, 'xml')
    names = soup.find_all('data')
    res = []
    for name in names:
        name = name.text.split(",")
        res.append([float(i) for i in name])
    res = np.array(res)

    # time exatraction 
    start_time = soup.find_all('startDateTimeIndex')[0].text
    #time to unix epoch time  
    utc_time = datetime.strptime(start_time[0:-6],
                             "%Y-%m-%dT%H:%M:%S.%f")
    data[file_name] = [res,  utc_time.timestamp()]


In [ ]:
names = ["LAF","ST","AST","TMP"]
fig = go.Figure()
for num in range(4):
    fig.add_trace(go.Scatter(
                        y=data["channel 2_20230912104409796.xml"][0][:,num],
                        line=dict(width=2),
                        name = names[num]              
    ))
fig.update_layout(
    title=dict(text="data"),
    xaxis_title="step", yaxis_title="value"
)
fig.show()


### Запускай все что ниже для 3D и карты высот

### Построение зависимости глубины температуры и времени

In [ ]:
# Check all file names in data directory
dir_path = "C:\\Users\\picni\\OneDrive\\Рабочий стол\\Работа\\opto_ident\\Данные\\204\\Data"
files_indir_names = []
for path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, path)):
        files_indir_names.append(dir_path + "\\" + path)
temp = []
time = []
laf = []
# Iterate all files in data directory and add <data> information 
for file_name in tqdm(files_indir_names):
    with open(file_name, 'r') as f:
        file = f.read() 
    soup = BeautifulSoup(file, 'xml')
    names = soup.find_all('data')
    res = []
    for name in names:
        name = name.text.split(",")
        res.append([float(i) for i in name])
    res = np.array(res)
    temp.append(res[:,3])
    # time exatraction 
    start_time = soup.find_all('startDateTimeIndex')[0].text
    #time to unix epoch time  
    utc_time = datetime.strptime(start_time[0:-6],
                             "%Y-%m-%dT%H:%M:%S.%f")
    time.append(utc_time.timestamp())
    laf.append(res[:,0])



### Для сохранения в JSON

In [ ]:
json_dict = dict()
indx = 0
for i in time:
    json_dict[i] = {"temp": temp[indx].tolist(), "laf": laf[indx].tolist()}
    indx += 1
json_object = json.dumps(json_dict, indent = 4)
with open("data.json", "w") as outfile:
    outfile.write(json_object)

### Для считывания из JSON

In [ ]:
np.array(time)[0]

In [ ]:
with open('data.json', 'r') as openfile:
    json_object = json.load(openfile)
time = list(json_object.keys())
time = np.array([float(i) for i in time])
laf = np.array([i['laf'] for i in json_object.values()])
temp = np.array([i['temp'] for i in json_object.values()])


In [ ]:
# Прорежаем данные 
DELIM = 1
temp_new = temp[:,::DELIM]
laf_new = laf[0][::DELIM]

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Heatmap(
    z=temp_new,
    zmax=70,  
    zmin=20,
    # x = laf_new,
    y = np.array(time),
))
fig.update_layout(
    # xaxis_range=[30,1457]
)
# fig.write_html("file.html")

fig.show()

In [ ]:
time[310]

In [ ]:
def corr(x, y):
    x = (x - x.mean())
    x /= np.linalg.norm(x)
    y = (y - y.mean())
    y /= np.linalg.norm(y)
    
    result = np.correlate(x, y, mode='full')
    result = result[result.size//2:]
    index_max = np.argmax(result)
    return index_max

In [ ]:
offset = {}
ref_idx = 13450
for i in range(250):
    off_idx =  ref_idx - i
    x = temp[340:520, ref_idx]
    y = temp[340:520, off_idx]
    result = corr(y,x)
    offset[laf_new[off_idx]] = result
    import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(
    x =list(offset.keys()),
    y = np.array(list(offset.values()))*12*60,
))
fig.write_html("file.html")

fig.show()

In [ ]:
import plotly.graph_objects as go


fig = go.Figure()
for i in range(56):
    temp_new = temp[i,:]
    print(laf_new[i*10])

    fig.add_trace(go.Scatter(
        y = temp_new,
        name  = i*12*60*60,
        x = laf_new#[330:520]
    ))
fig.update_layout(
    # xaxis_range=[30,1457]
)
fig.write_html("file.html")

fig.show()

### Фильтр Лапласа